In [1]:
from itertools import islice


def to_batches(it, n):
    it = iter(it)
    return iter(lambda: tuple(islice(it, n)), ())

In [9]:
import torch
import trimesh
import torch.nn.functional as F
from meshae.model import MeshAEFeatEmbedConfig, MeshAEModel


mesh = trimesh.load_mesh(
    "../data/objaverse/train/0aa9882e169a4f228a27a1a88126ca78.glb",
    file_type="glb",
)
vertices = torch.from_numpy(mesh.vertices).unsqueeze(0)
faces = torch.from_numpy(mesh.faces).unsqueeze(0)
face_masks = torch.ones((1, faces.size(1))).bool()

In [2]:
faces = faces.masked_fill(~face_masks.unsqueeze(-1), 0)
index = torch.arange(vertices.size(0), device=vertices.device)[:, None, None]

coords = vertices[index, faces]

In [11]:
offsets = F.pad(faces.amax(dim=(1, 2)).cumsum(0).int() + 1, (1, -1), value=0)
offsets

tensor([0], dtype=torch.int32)

In [15]:
(faces.amax(dim=(1, 2)).int() + 1).sum().item()

696

In [8]:
vertices.shape

torch.Size([1, 696, 3])

In [19]:
face_embeds = model.embedding(coords, face_masks, None, None)

KeyError: 'norm'

In [ ]:

feat_configs = {
    "vertex": MeshAEFeatEmbedConfig(),
    "norm": MeshAEFeatEmbedConfig(),
}
model = MeshAEModel(feat_configs, num_sageconv_layers=0, num_quantizers=2)

In [9]:
model.embedding

MeshAEEmbedding(
  (embeddings): ModuleDict(
    (vertex): Embedding(128, 128)
  )
  (proj_in): Sequential(
    (0): Linear(in_features=1152, out_features=512, bias=True)
    (1): GELU(approximate='none')
  )
)

In [8]:
model.encoder.quantizer.num_quantizers

2